<a href="https://colab.research.google.com/github/chloebh9/Recommendation-system/blob/main/L10_1_Personalized_Models_of_Visual_Data_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  CLIP

In [1]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [2]:
url1 = "https://img.etoday.co.kr/pto_db/2024/04/600/20240419110132_2014644_647_358.jpg"
url2 = "https://t1.daumcdn.net/brunch/service/user/cnoC/image/Gy88dug6GPgiMTpWUBPD42On4uM.JPG"
image1 = Image.open(requests.get(url1, stream=True).raw)
image2 = Image.open(requests.get(url2, stream=True).raw)
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=[image1, image2], return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score

probs = logits_per_image.softmax(dim=1)
print(probs)

tensor([[0.9526, 0.0474],
        [0.0077, 0.9923]], grad_fn=<SoftmaxBackward0>)


In [3]:
print(inputs.keys())
# dict_keys(['input_ids', 'attention_mask', 'pixel_values'])
# input_ids : 텍스트를 토크나이즈해서 integer로 바꿔주는 애
# attention_mask : 패딩된 결과에서 어디까지가 실제 내용이고 어디가 패딩된 곳인지 알려줌
# pixel_values : 224 * 224로 이미지 축소도 하고 채널은 3개 RGB, 그리고 이미지가 2개니까 이렇게 됨 -> 아래 실행결과

print(outputs.keys())
# odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])
# logits_per_image :
# logits_per_text :
# text_embeds : 텍스트 인코더를 거쳐서 나오는 latent vector가 CLIP모델에 안맞아서 차원에 맞게끔 매트릭스를 한번 더 곱함 - 최종 임베딩
# image_embeds : 최종 임베딩
# text_model_output : 중간 임베딩
# vision_model_output : 중간 임베딩

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])
odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])


In [4]:
inputs['pixel_values'].shape

torch.Size([2, 3, 224, 224])

In [5]:
print(outputs['logits_per_image'])
# 결과에 1열이 고양이, 2열이 강아지에 대한 확률임을 나타냄
print(outputs['logits_per_image'].softmax(dim=1))

tensor([[28.4896, 25.4883],
        [24.1119, 28.9683]], grad_fn=<TBackward0>)
tensor([[0.9526, 0.0474],
        [0.0077, 0.9923]], grad_fn=<SoftmaxBackward0>)


##  Estimating Compatibility from Co-purchases

In [6]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/pml_data/image_features_Baby.b
!wget https://datarepo.eng.ucsd.edu/mcauley_group/pml_data/meta_Baby.json.gz
!gunzip meta_Baby.json.gz

--2024-06-02 13:35:20--  https://datarepo.eng.ucsd.edu/mcauley_group/pml_data/image_features_Baby.b
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1159318104 (1.1G) [chemical/x-molconn-Z]
Saving to: ‘image_features_Baby.b’

image_features_Baby 100%[===================>]   1.08G  9.85MB/s    in 23m 12s 

2024-06-02 13:58:38 (814 KB/s) - ‘image_features_Baby.b’ saved [1159318104/1159318104]

--2024-06-02 13:58:38--  https://datarepo.eng.ucsd.edu/mcauley_group/pml_data/meta_Baby.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31328004 (30M) [application/x-gzip]
Saving to: ‘meta_Baby.json.gz’

meta_Baby.json.gz   100%[===================>]  29.8

### Image feature 불러오기

In [7]:
import torch
device = torch.device("cuda")

In [8]:
import numpy as np

asin_to_pid = {}
pid_to_asin = []
features = []

with open("image_features_Baby.b", "rb") as f:
    while True:
        asin = f.read(10).decode('utf-8')
        if len(asin) < 10: break
        a = np.fromfile(f, dtype=np.float32, count=4096)

        pid_to_asin.append(asin)
        asin_to_pid[asin] = len(asin_to_pid)

        features.append(a)
features = torch.tensor(features).to(device)

<ipython-input-8-571bd8fdb977>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  features = torch.tensor(features).to(device)


In [9]:
print(asin_to_pid)

{'B000EE3HB6': 0, 'B000BTTPOW': 1, 'B000ETLO0M': 2, 'B000BM4OFE': 3, 'B000GB0O0Y': 4, 'B000FR9AI6': 5, 'B000C4W6UG': 6, 'B000BTPTTW': 7, 'B000BPMJS0': 8, 'B000C29JYE': 9, 'B000FNXVDA': 10, 'B000FI64DE': 11, 'B000GIBYQ0': 12, 'B000F6R3TU': 13, 'B000F1OMMQ': 14, 'B000F29252': 15, 'B0007WWZW2': 16, 'B000GTYMIQ': 17, 'B0007Y3KRE': 18, 'B000G7YLUW': 19, 'B000G80OK2': 20, 'B000G7YRDI': 21, 'B000EF5INK': 22, 'B000BVMGI2': 23, 'B000G1C3IK': 24, 'B000F1SC2C': 25, 'B000FT7NYC': 26, 'B000BF2VO2': 27, 'B000GV702E': 28, 'B000BPS0HE': 29, 'B000FOLJY2': 30, 'B000G82GO4': 31, 'B000BKHYQW': 32, 'B000GQ1RTG': 33, 'B0007US56Y': 34, 'B000BPS0QA': 35, 'B000BNRDJ2': 36, 'B0007TPYUK': 37, 'B000FP67SY': 38, 'B000F1QSAK': 39, 'B000BUTCOO': 40, 'B000HGBTLG': 41, 'B000HHHDUG': 42, 'B000F1QJU4': 43, 'B000F1MT4O': 44, 'B000H8ESQC': 45, 'B000G82HPM': 46, 'B000GAYYBK': 47, 'B000FKOX6C': 48, 'B000FT7NQA': 49, 'B000E9T93G': 50, 'B0007CQ6SG': 51, 'B000FP7M6K': 52, 'B000BRGIW6': 53, 'B000F1MD7C': 54, 'B000BQKWHE': 55, '

###  meta_Baby.json 불러오기

In [10]:
from torch.utils.data import TensorDataset, DataLoader

compat = []
imurls = {}

with open("meta_Baby.json", "r") as f:
    for line in f:
        record = eval(line)
        if record['asin'] not in asin_to_pid: continue

        asin1 = record['asin']
        pid1 = asin_to_pid[asin1]
        imurls[asin1] = record['imUrl']

        if 'related' not in record or 'also_bought' not in record['related']: continue

        for asin2 in record['related']['also_bought']:
            if asin2 in asin_to_pid:
                pid2 = asin_to_pid[asin2]
                compat.append( (pid1, pid2) ) # 두 아이템이 연관됨을 알 수 있음

compat = torch.LongTensor(compat)
compat_dataset = TensorDataset(compat)
dataloader = DataLoader(compat_dataset, batch_size=65536, shuffle=True)
# batch_size : 한꺼번에 가져올 데이터의 개수

In [11]:
from torch import nn
dim_feat = 4096
dim_style = 5

c = torch.zeros(1, requires_grad=True, device=device)
E1 = torch.randn(dim_feat, dim_style, requires_grad=True, device=device) #4096*5짜리가 필요하니까 이렇게 만듦.
E2 = torch.randn(dim_feat, dim_style, requires_grad=True, device=device)

In [12]:
E1.shape

torch.Size([4096, 5])

In [13]:
optim = torch.optim.Adam([E1, E2, c], lr=0.1)
logsig = nn.LogSigmoid()

for epoch in range(31):
    for [pos] in dataloader:
        neg = torch.randint_like(pos, len(asin_to_pid))

        si = features[pos[:,0]] @ E1
        sj = features[pos[:,1]] @ E2
        pos_obj = logsig(c - ((si - sj) ** 2).sum(dim=1)).sum()

        si = features[neg[:,0]] @ E1
        sj = features[neg[:,1]] @ E2
        neg_obj = logsig(-c + ((si - sj) ** 2).sum(dim=1)).sum()

        cost = -(pos_obj + neg_obj)

        optim.zero_grad()
        cost.backward()
        optim.step()

    with torch.no_grad():
        print(epoch, cost.item())

0 2595945984.0
1 1922702848.0
2 958475904.0
3 791397376.0
4 665781248.0
5 577285760.0
6 513447168.0
7 459465920.0
8 416169984.0
9 381055808.0
10 345771968.0
11 317355616.0
12 294745376.0
13 269589408.0
14 249784624.0
15 233059456.0
16 216849600.0
17 202812512.0
18 188949440.0
19 176449584.0
20 165091168.0
21 155479712.0
22 145241120.0
23 137383840.0
24 128905816.0
25 121474200.0
26 114767080.0
27 108843672.0
28 103044984.0
29 95711288.0
30 91757120.0


### ‘가까운’ 아이템 확인

In [20]:
pid = 100

si = features[914].view(1,-1) @ E1
sj = features @ E2
objs = (((si - sj) ** 2).sum(dim=1)).detach().tolist()
res = sorted(enumerate(objs), key=lambda x: x[1])[:10]

print("query:")
print(imurls[pid_to_asin[pid]])
print("results:")

for a, b in res:
    print(imurls[pid_to_asin[a]])

query:
http://ecx.images-amazon.com/images/I/41z9FObnNML._SX300_.jpg
results:
http://ecx.images-amazon.com/images/I/41jx8H0CifL._SY300_.jpg
http://ecx.images-amazon.com/images/I/41VDhoaQ3rL._SX300_.jpg
http://ecx.images-amazon.com/images/I/41b%2BRpWXMYL._SY300_.jpg
http://ecx.images-amazon.com/images/I/418h35jJ8gL._SY300_.jpg
http://ecx.images-amazon.com/images/I/31VrRTof2KL._SY300_.jpg
http://ecx.images-amazon.com/images/I/41vw5BuD9eL._SY300_.jpg
http://ecx.images-amazon.com/images/I/41HJxeq19vL._SY300_.jpg
http://ecx.images-amazon.com/images/I/51rTAgLJz4L._SY300_.jpg
http://ecx.images-amazon.com/images/I/513plIbCLoL._SX300_.jpg
http://ecx.images-amazon.com/images/I/51EvuJ00PyL._SY300_.jpg
